# ImageNet-C

In [1]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from IPython.display import display

CATEGORIES = ['imagenet', 'noise', 'blur', 'weather', 'digital', 'extra']
COLORS = ['Greys', 'Oranges', 'Blues', 'Reds', 'Greens', 'Purples']
MCE_CATEGORIES = ['noise', 'blur', 'weather', 'digital']

def color_columns(df_styler):
    for category, color in zip(CATEGORIES, COLORS):
        subset = [cat == category for cat, *_ in df_styler.columns]
        df_styler = df_styler.background_gradient(axis=0, cmap=color, subset=subset)
    subset = [cat == 'mCE' for cat, *_ in df_styler.columns]
    df_styler = df_styler.text_gradient(axis=0, cmap='YlOrRd', subset=subset)
    return df_styler

## Baselines

In [2]:
df_baselines = pd.read_json('results/imagenet-c-baselines.json', lines=True)
df_baselines = df_baselines.query('corruption_category != "extra" or corruption_type != "style"')
df_baselines = (
    df_baselines
    .pivot_table(index='model', columns=['corruption_category', 'corruption_type'], values='error', sort=False)
    [CATEGORIES]
)
display(
    color_columns(df_baselines.style)
    .set_caption('Baselines top-1 error')
    .format('{:.1f}')
)

In [3]:
df_baselines_mce = 100 * df_baselines[MCE_CATEGORIES] / df_baselines.loc['AlexNet', MCE_CATEGORIES]
df_baselines_mce['mCE'] = df_baselines_mce.mean(axis=1)
display(
    color_columns(df_baselines_mce.style)
    .set_caption('Baselines mCE')
    .format('{:.1f}')
)

## Results

In [4]:
# Load results
df = []
for p in Path('results').glob('*/results.json'):
    for l in p.read_text().splitlines():
        df.append(json.loads(l))

df = pd.DataFrame(df)

# Computed columns
df['error'] = 100 - df['acc1']
df['size'] = df['model'].str.split('_').str.get(1)
df['activation'] = df['model'].str.split('LS').str.get(2).fillna('softmax')
df['corruption_category'] = df['data_path'].str.split('/').str.get(-3)
df['corruption_type'] = df['data_path'].str.split('/').str.get(-2)
df['corruption_level'] = df['data_path'].str.split('/').str.get(-1).astype(int)

df

,model,resume,data_set,data_path,num_samples,loss,acc1,acc5,error,size,activation,corruption_category,corruption_type,corruption_level
0,deit_medium_patch16_LS,checkpoints/deit_medium_patch16_LS.pth,FOLDER,blur/defocus_blur/1,50000,1.209056,73.332002,91.650003,26.667998,medium,softmax,blur,defocus_blur,1
1,deit_medium_patch16_LS,checkpoints/deit_medium_patch16_LS.pth,FOLDER,blur/defocus_blur/2,50000,1.409536,69.446002,88.970003,30.553998,medium,softmax,blur,defocus_blur,2
2,deit_medium_patch16_LS,checkpoints/deit_medium_patch16_LS.pth,FOLDER,blur/defocus_blur/3,50000,2.039976,57.756002,79.898003,42.243998,medium,softmax,blur,defocus_blur,3
3,deit_medium_patch16_LS,checkpoints/deit_medium_patch16_LS.pth,FOLDER,blur/defocus_blur/4,50000,2.777189,45.430001,68.076002,54.569999,medium,softmax,blur,defocus_blur,4
4,deit_medium_patch16_LS,checkpoints/deit_medium_patch16_LS.pth,FOLDER,blur/defocus_blur/5,50000,3.556627,33.872001,55.288002,66.127999,medium,softmax,blur,defocus_blur,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,deit_small_patch16_LS,checkpoints/deit_small_patch16_LS.pth,FOLDER,data/extra/saturate/1,50000,1.068155,77.346002,93.406003,22.653998,small,softmax,extra,saturate,1
284,deit_small_patch16_LS,checkpoints/deit_small_patch16_LS.pth,FOLDER,data/extra/saturate/2,50000,1.169473,75.244002,91.456003,24.755998,small,softmax,extra,saturate,2
285,deit_small_patch16_LS,checkpoints/deit_small_patch16_LS.pth,FOLDER,data/extra/saturate/3,50000,1.008706,78.618003,94.182002,21.381997,small,softmax,extra,saturate,3
286,deit_small_patch16_LS,checkpoints/deit_small_patch16_LS.pth,FOLDER,data/extra/saturate/4,50000,1.202601,74.824003,92.030003,25.175997,small,softmax,extra,saturate,4


In [5]:
df_counts = (
    df.groupby(['activation', 'size', 'corruption_category', 'corruption_type'])
    ['corruption_level'].nunique()
    .unstack(['corruption_category', 'corruption_type'])
    .fillna(0, downcast='infer')
    [CATEGORIES]
)

display(
    color_columns(df_counts.style)
    .set_caption('How many subset have been evaluated?')
)

In [6]:
df_error = (
    df.groupby(['activation', 'size', 'corruption_category', 'corruption_type'])
    ['error'].mean()
    .unstack(['corruption_category', 'corruption_type'])
    [CATEGORIES]
)

display(
    color_columns(df_error.style)
    .set_caption('Average top-1 error')
    .format('{:.1f}')
)

In [7]:
df_error[MCE_CATEGORIES]

corruption_category          noise                                  blur  \
corruption_type     gaussian_noise impulse_noise shot_noise defocus_blur   
activation size                                                            
softmax    base          29.485198     30.767198  31.101598    40.537998   
           medium        32.213198     33.605598  33.579198    44.032798   
           small         37.686398     39.613998  40.365599    46.354399   

corruption_category                                      weather             \
corruption_type     glass_blur motion_blur  zoom_blur brightness        fog   
activation size                                                               
softmax    base      49.167198   36.343598  48.887599  20.517997  26.551198   
           medium    54.296799   39.434798  52.119999  21.782397  27.745598   
           small     56.477199   43.117998  57.023199  23.140798  29.043198   

corruption_category                          digital                    \
corruption_type          frost       snow   contrast elastic_transform   
activation size                                                          
softmax    base      31.343598  33.263598  26.781998         38.548798   
           medium    33.241598  36.054798  28.717198         41.936398   
           small     34.892398  38.827199  30.775998         45.096799   

corruption_category                              
corruption_type     jpeg_compression   pixelate  
activation size                                  
softmax    base            29.608798  30.935998  
           medium          31.496398  33.260398  
           small           34.391198  35.765198

In [8]:
df_mce = 100 * df_error[MCE_CATEGORIES] / df_baselines.loc['AlexNet', MCE_CATEGORIES]
df_mce = df_mce[MCE_CATEGORIES]
df_mce['mCE'] = df_mce.mean(axis=1)
display(
    color_columns(df_mce.style)
    .set_caption('mCE')
    .format('{:.1f}')
)